# Pantheon Data Collection

In [1]:
import sys,os,glob
from astropy.io import fits
from astropy.table import Table
from astropy.nddata import extract_array
from astropy.coordinates import SkyCoord
from astropy import wcs
from astropy.wcs.utils import skycoord_to_pixel
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt
from astroquery.mast import Observations
from astropy.visualization import (simple_norm,LinearStretch)
import pandas as pd
import space_phot



The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    


In [2]:
#Open and read the unmatched pantheon dataset
df = pd.read_csv('unmatched_pantheon(in).csv')

print(df.dtypes)
df_string = df.astype(str)
print(df_string.dtypes)

SNID           object
IAUC           object
host           object
RA            float64
Dec           float64
RA_host       float64
Dec_host      float64
zhel          float64
zcmb          float64
zhelerr       float64
zHD           float64
zHDerr        float64
PV              int64
vpecerr         int64
RA_group      float64
Dec_group     float64
zhel_group    float64
zcmb_group    float64
zHD_group     float64
PV_group      float64
in_group        int64
has_host        int64
is_SNz          int64
dtype: object
SNID          object
IAUC          object
host          object
RA            object
Dec           object
RA_host       object
Dec_host      object
zhel          object
zcmb          object
zhelerr       object
zHD           object
zHDerr        object
PV            object
vpecerr       object
RA_group      object
Dec_group     object
zhel_group    object
zcmb_group    object
zHD_group     object
PV_group      object
in_group      object
has_host      object
is_SNz        obje

In [3]:
#MAST takes RA and Dec as a string, creating the string representation of each supernovae in a new column called "resolved_coord"
df = df.assign(resolved_coord = df_string['RA'] + " " + df_string['Dec'])
print(df['resolved_coord'])

0         167.62425 55.16083
1         248.61546 76.02967
2          358.38658 8.11739
3          21.34542 34.02514
4         144.48792 25.49481
                ...         
1856     53.259174 -26.85409
1857     52.695011 -28.64933
1858    54.343502 -28.826429
1859     55.340694 -29.14934
1860    52.889896 -28.912092
Name: resolved_coord, Length: 1861, dtype: object


In [4]:
#Testing the observation table prior to creating a query
i = 0
print(df['resolved_coord'][i]) 

obs_table = Observations.query_criteria(coordinates=df['resolved_coord'][i],
                                        radius="0.1 deg",
                                        intentType = 'science',
                                        obs_collection=['HST', 'JWST'])
print(obs_table)

167.62425 55.16083
intentType obs_collection provenance_name ... objID objID1 distance
---------- -------------- --------------- ... ----- ------ --------


In [5]:
#Creating a function that will create a folder for the data (named the resolved coordinate) and keep the query there
def save_images_supernova(resolved_coord_index):
    """
    If there are images for the supernovae in JWST or HST at the resolved query, create a folder and keep the data there
    Parameters:
        resolved_coord_index: integer, the index number of the query
    Returns:
        Folder of Data at requested file path (pantheon_data_folder/{resolved coord})
    """
    print(resolved_coord_index, df['resolved_coord'][resolved_coord_index])

    #try the the query, if a error is thrown then print "No Data Points" and "skip" this data point, else make folder and store data
    try:
        obs_table = Observations.query_criteria(coordinates=df['resolved_coord'][resolved_coord_index],
                                            radius="0.006 deg",
                                            intentType = 'science',
                                            obs_collection=['HST', 'JWST'])
        data_products = Observations.get_product_list(obs_table)
    except: 
        print("No Data Points")
        exit
    else:
        folder_path = "pantheon_data_folder/{}".format(df['resolved_coord'][resolved_coord_index])
        os.makedirs(folder_path, exist_ok=True)
        #Fits is what we need for space_phot, jpg to see image
        manifest = Observations.download_products(data_products, download_dir=folder_path, extension=['fits', 'jpg'])
        print(manifest)

In [ ]:
for i in range(len(df['resolved_coord'])):
    save_images_supernova(i)

10 216.16883 26.62631


2025-05-29 11:30:40,714 - stpipe - INFO - 122 of 203 products were duplicates. Only returning 81 unique product(s).
2025-05-29 11:30:40,734 - stpipe - INFO - Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f555w_pc_01/hst_07281_37_wfpc2_f555w_pc_01_drz.fits with expected size 4443840.
2025-05-29 11:30:40,792 - stpipe - INFO - Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f555w_pc/hst_07281_37_wfpc2_f555w_pc_drz.fits with expected size 13296960.
2025-05-29 11:30:40,838 - stpipe - INFO - Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f555w_wf_01/hst_07281_37_wfpc2_f555w_wf_01_drz.fits with expected size 18532800.
2025-05-29 11:30:40,897 - stpipe - INFO - Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f555w_wf/hst_07281_37_wfpc2_f555w_wf_drz.fits with expected size 55543680.


INFO: 122 of 203 products were duplicates. Only returning 81 unique product(s). [astroquery.mast.utils]
INFO: Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f555w_pc_01/hst_07281_37_wfpc2_f555w_pc_01_drz.fits with expected size 4443840. [astroquery.query]
INFO: Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f555w_pc/hst_07281_37_wfpc2_f555w_pc_drz.fits with expected size 13296960. [astroquery.query]
INFO: Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f555w_wf_01/hst_07281_37_wfpc2_f555w_wf_01_drz.fits with expected size 18532800. [astroquery.query]
INFO: Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f555w_wf/hst_07281_37_wfpc2_f555w_wf_drz.fits with expected size 55543680. [astroquery.query]


2025-05-29 11:30:40,958 - stpipe - INFO - Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f814w_pc_01/hst_07281_37_wfpc2_f814w_pc_01_drz.fits with expected size 4443840.
2025-05-29 11:30:41,000 - stpipe - INFO - Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f814w_pc/hst_07281_37_wfpc2_f814w_pc_drz.fits with expected size 13296960.
2025-05-29 11:30:41,056 - stpipe - INFO - Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f814w_wf_01/hst_07281_37_wfpc2_f814w_wf_01_drz.fits with expected size 18532800.
2025-05-29 11:30:41,096 - stpipe - INFO - Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f814w_wf/hst_07281_37_wfpc2_f814w_wf_drz.fits with expected size 55543680.
2025-05-29 11:30:41,135 - stpipe - INFO - Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_total_wf/

INFO: Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f814w_pc_01/hst_07281_37_wfpc2_f814w_pc_01_drz.fits with expected size 4443840. [astroquery.query]
INFO: Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f814w_pc/hst_07281_37_wfpc2_f814w_pc_drz.fits with expected size 13296960. [astroquery.query]
INFO: Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f814w_wf_01/hst_07281_37_wfpc2_f814w_wf_01_drz.fits with expected size 18532800. [astroquery.query]
INFO: Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f814w_wf/hst_07281_37_wfpc2_f814w_wf_drz.fits with expected size 55543680. [astroquery.query]
INFO: Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_total_wf/hst_07281_37_wfpc2_total_wf_drz.fits with expected size 55543680. [astroquery.query]


...

2025-05-29 11:30:46,365 - stpipe - INFO - Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HST/u3ws3702r/u3ws3702r_x0m.fits with expected size 109440.


INFO: Found cached file pantheon_data_folder/216.16883 26.62631/mastDownload/HST/u3ws3702r/u3ws3702r_x0m.fits with expected size 109440. [astroquery.query]
                                                           Local Path                                                           ...
------------------------------------------------------------------------------------------------------------------------------- ...
pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f555w_pc_01/hst_07281_37_wfpc2_f555w_pc_01_drz.fits ...
      pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f555w_pc/hst_07281_37_wfpc2_f555w_pc_drz.fits ...
pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f555w_wf_01/hst_07281_37_wfpc2_f555w_wf_01_drz.fits ...
      pantheon_data_folder/216.16883 26.62631/mastDownload/HLA/hst_07281_37_wfpc2_f555w_wf/hst_07281_37_wfpc2_f555w_wf_drz.fits ...
pantheon_data_folder/216.16883 26.62631/mastDownload

2025-05-29 11:31:30,341 - stpipe - INFO - 19606 of 24737 products were duplicates. Only returning 5131 unique product(s).


INFO: 19606 of 24737 products were duplicates. Only returning 5131 unique product(s). [astroquery.mast.utils]


KeyboardInterrupt: 

In [ ]:
def HST_aperture(resolved_coordinate_index):
    """
    For HST images, use the aperature techinique to go through and find PSF photometry
    Parameters:
        resolved_coordinate_index: integer, the index number of the query
    Returns:
        - Full image of the field
        - Image of the supernova
        - Print final aperture result
    """
    RA = df['RA'][resolved_coordinate_index]
    DEC = df['Dec'][resolved_coordinate_index]
    resolved_coord = df['resolved_coord'][resolved_coordinate_index]

    files = glob.glob("pantheon_data_folder/{}/mastDownload/HST/*/*flt.fits".format(resolved_coord))
    #If there are NO HST IMAGES exit
    if len(files) == 0:
        print("No HST images")
        exit

    #RIGHT NOW THIS IS 0 THIS WILL NEED TO CHANGE
    image_num = 0

    #Takes the data from the file

    ref_image = files[image_num]
    ref_fits = fits.open(ref_image)
    ref_data = fits.open(ref_image)['SCI',1].data

    #Creating the visual of the image region
    norm1 = simple_norm(ref_data,stretch='linear',min_cut=-1,max_cut=10)
    plt.imshow(ref_data, origin='lower',norm=norm1,cmap='gray')
    plt.gca().tick_params(labelcolor='none',axis='both',color='none')
    plt.title("Image of Full File {} for coord {}".format(image_num, resolved_coord))
    plt.show()


    #Taking the supernova position and zooming in on this image + creating image
    sn_location = SkyCoord(ra=RA*u.degree, dec=DEC*u.degree, frame='icrs')
    ref_y,ref_x = skycoord_to_pixel(sn_location,wcs.WCS(ref_fits['SCI',1],ref_fits))

    #400 by 400 image
    full_cutout = extract_array(ref_data,(400,400),(ref_x,ref_y))
    ref_cutout = extract_array(ref_data,(11,11),(ref_x,ref_y))
    norm1 = simple_norm(full_cutout,stretch='linear',vmin=-1,vmax=10)
    #Image in black and grey
    plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
    #Target Region over it in Red
    plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='reds')
    plt.title("400 by 400 {}".format(resolved_coord))
    plt.gca().tick_params(labelcolor='none',axis='both',color='none')
    plt.show()

    #more Zoomed in
    ref_cutout = extract_array(ref_data,(11,11),(ref_x,ref_y))
    norm1 = simple_norm(ref_cutout,stretch='linear',vmin=-1,vmax=10)
    plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
    plt.title(resolved_coord)
    plt.gca().tick_params(labelcolor='none',axis='both',color='none')
    plt.show()
    print (files)

    #Final Aperature result
    hst_obs = space_phot.observation2(files)
    hst_obs.aperture_photometry(sn_location,radius=3,
                        skyan_in=5,skyan_out=7)
    print(hst_obs.aperture_result.phot_cal_table)


In [ ]:
#Create a function that will take the index of the resolved coordinate index and do analysis on the files in that subgroup

def HST_PSF_function(resolved_coordinate_index):
    """
    For Hubble Space Telescope (HST) images, go through and find fluxes of the supernova after cleaning (PSF)
    Params:
        resolved_coordinate_index: integer, the index number of the query
    Returns:
        - Full image of the field
        - Image of the supernova
        - PSF model
        - PSF photometry overall
    """
    
    RA = df['RA'][resolved_coordinate_index]
    DEC = df['Dec'][resolved_coordinate_index]
    resolved_coord = df['resolved_coord'][resolved_coordinate_index]

    files = glob.glob("pantheon_data_folder/{}/mastDownload/HST/*/*flt.fits".format(resolved_coord))
    #If there are NO HST IMAGES exit
    if len(files) == 0:
        print("No HST images")
        exit
    
    #RIGHT NOW THIS IS 0 THIS WILL NEED TO CHANGE
    image_num = 0

    #Takes the data from the file

    ref_image = files[image_num]
    ref_fits = fits.open(ref_image)
    ref_data = fits.open(ref_image)['SCI',1].data

    #Creating the visual of the image region
    norm1 = simple_norm(ref_data,stretch='linear',min_cut=-1,max_cut=10)
    plt.imshow(ref_data, origin='lower',norm=norm1,cmap='gray')
    plt.gca().tick_params(labelcolor='none',axis='both',color='none')
    plt.title("Image of File {} for coord {}".format(image_num, resolved_coord))
    plt.show()

    #Taking the supernova position and zooming in on this image + creating image
    sn_location = SkyCoord(ra=RA*u.degree, dec=DEC*u.degree, frame='icrs')
    ref_y,ref_x = skycoord_to_pixel(sn_location,wcs.WCS(ref_fits['SCI',1],ref_fits))
    ref_cutout = extract_array(ref_data,(11,11),(ref_x,ref_y))
    norm1 = simple_norm(ref_cutout,stretch='linear',vmin=-1,vmax=10)
    plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
    plt.title(resolved_coord)
    plt.gca().tick_params(labelcolor='none',axis='both',color='none')
    plt.show()
    print (files)
    """
    hst_obs = space_phot.observation2(files)
    print(hst_obs)
    psfs = space_phot.get_hst_psf(hst_obs,sn_location)
    plt.imshow(psfs[0].data)
    plt.show()
    """


    

HST_PSF_function(2)


In [ ]:
files = glob.glob('pantheon_data_folder/*/mastDownload/HST/*/*flt.fits')
print(len(files))

ref_image = files[0]
ref_fits = fits.open(ref_image)
ref_data = fits.open(ref_image)['SCI',1].data
norm1 = simple_norm(ref_data,stretch='linear',min_cut=-1,max_cut=10)

plt.imshow(ref_data, origin='lower',norm=norm1,cmap='gray')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()

In [ ]:
sn_location = SkyCoord('21:29:40.2110','+0:05:24.154',unit=(u.hourangle,u.deg))
ref_y,ref_x = skycoord_to_pixel(sn_location,wcs.WCS(ref_fits['SCI',1],ref_fits))
ref_cutout = extract_array(ref_data,(11,11),(ref_x,ref_y))
norm1 = simple_norm(ref_cutout,stretch='linear',min_cut=-1,max_cut=10)
plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
plt.title('SN2022riv')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()

In [ ]:
resolved_coordinate_index = 2
RA = df['RA'][resolved_coordinate_index]
DEC = df['Dec'][resolved_coordinate_index]
resolved_coord = df['resolved_coord'][resolved_coordinate_index]

files = glob.glob("pantheon_data_folder/{}/mastDownload/HST/*/*flt.fits".format(resolved_coord))

print(len(files))
#If there are NO HST IMAGES exit


#RIGHT NOW THIS IS 0 THIS WILL NEED TO CHANGE
ref_image = files[0]
ref_fits = fits.open(ref_image)
ref_data = fits.open(ref_image)['SCI',1].data
norm1 = simple_norm(ref_data,stretch='linear',min_cut=-1,max_cut=10)

plt.imshow(ref_data, origin='lower',norm=norm1,cmap='gray')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()
"""
sn_location = SkyCoord(ra=RA*u.degree, dec=DEC*u.degree, frame='icrs')
ref_y,ref_x = skycoord_to_pixel(sn_location,wcs.WCS(ref_fits['SCI',1],ref_fits)) # pyright: ignore[reportUndefinedVariable]
ref_cutout = extract_array(ref_data,(11,11),(ref_x,ref_y))
norm1 = simple_norm(ref_cutout,stretch='linear',vmin=-1,vmax=10)
plt.imshow(ref_cutout, origin='lower',
                    norm=norm1,cmap='gray')
plt.title(resolved_coord)
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()
"""